# IMPORT LIBRARIES

In [9]:
import pandas as pd
from scipy import stats
import BiQ_Common_Fxns as BiQ
import pandas as pd
from scipy import stats
from tkinter import filedialog, Tk, simpledialog
import tkinter as tk

# EXTRACT DATA

In [10]:
def read_excel_file(file_path):
    """Reads an Excel file and returns a DataFrame."""
    try:
        df = pd.read_excel(file_path)  # Read Excel file into a temporary DataFrame
        df = df.dropna(axis=1, how='all')  # Drop columns that are entirely blank
        return df  # Return the DataFrame
    except Exception as e:
        print(f"Error reading Excel file: {e}")  # Print error message if file reading fails
        return None  # Return None if an error occurs

def perform_t_test(group1, group2):
    """Performs a two-sample t-test and prints the results."""
    try:
        # Perform two-sample t-test
        t_stat, p_value = stats.ttest_ind(group1, group2)

        # Print results
        print(f"\nt-statistic: {t_stat}")
        print(f"p-value: {p_value}")

        # Interpret the results
        alpha = 0.05  # significance level
        if p_value < alpha:
            print("Reject the null hypothesis: There is a significant difference between the groups.")
        else:
            print("Fail to reject the null hypothesis: There is no significant difference between the groups.")
    except Exception as e:
        print(f"Error performing t-test: {e}")

def get_user_column_selection(df):
    """Gets user input for selecting columns using dialog boxes."""
    root = Tk()
    root.withdraw()  # Hide the main window

    # Get column names from the DataFrame
    columns = df.columns.tolist()

    # Ask user to select columns for the two groups
    group1_col = simpledialog.askstring("Input", f"Enter the column name for group 1:\nAvailable columns: {', '.join(columns)}")
    group2_col = simpledialog.askstring("Input", f"Enter the column name for group 2:\nAvailable columns: {', '.join(columns)}")

    return group1_col, group2_col

def main():
    # Open a file dialog to select Excel file
    root = Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
    if not file_path:
        print("No file selected.")
        return

    # Read Excel file into DataFrame
    df = read_excel_file(file_path)
    if df is None:
        print("Error reading Excel file.")
        return

    print("Columns in Excel sheet:")
    print(df.columns)  # Display column names

    # Get user input for column selection
    group1_col, group2_col = get_user_column_selection(df)

    # Check if the selected columns exist in the DataFrame
    if group1_col in df.columns and group2_col in df.columns:
        # Extract data from selected columns
        group1 = df[group1_col].dropna()
        group2 = df[group2_col].dropna()

        # Print the selected groups as columns
        print(f"\nSelected Group 1 ({group1_col}):\n{group1.to_frame()}")
        print(f"\nSelected Group 2 ({group2_col}):\n{group2.to_frame()}")

        # Perform two-sample t-test
        perform_t_test(group1, group2)
    else:
        print("Error: One or both selected columns do not exist in the DataFrame.")

if __name__ == "__main__":
    main()

Columns in Excel sheet:
Index(['X', 'Y'], dtype='object')

Selected Group 1 (X):
     X
0    2
1    7
2   14
3    5
4   19
5   12
6   18
7    7
8    6
9    5
10   6
11   7
12   6
13  13
14  20
15   9
16   4
17  18
18   4
19   2
20   4
21  17
22   1
23   3
24   4
25   9
26  11
27  17
28   2
29  19
30   7
31  16
32  12
33  10
34  15
35   6
36   6
37  19
38  19
39  16
40  18
41  18
42   5
43  11
44  10
45   8
46  16
47   1
48  17

Selected Group 2 (Y):
     Y
0    1
1    5
2   19
3   12
4    8
5   17
6   17
7    4
8   11
9    9
10   5
11  16
12   5
13   2
14   2
15   6
16  19
17  15
18   5
19   4
20   2
21  17
22   1
23  12
24  12
25   2
26   6
27  14
28  17
29  15
30  13
31   3
32   8
33   5
34  14
35   8
36   3
37   1
38   6
39  20
40  20
41  20
42  20
43  12
44  13
45   7
46  10
47  16
48   8

t-statistic: 0.23251665341537137
p-value: 0.81663174152102
Fail to reject the null hypothesis: There is no significant difference between the groups.
